In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import os
import gc
import glob
import numba
import numpy.linalg as la
from scipy import stats
from scipy.stats import circmean
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist

In [2]:
foldername = 'ffA3'
n_inds = 10

window_size = '01' # sliding window size in seconds
fps = 10

In [16]:
df = pd.read_csv('/home/user/Documents/Vivek/cuda/DirectionalCorrelation/Data/Output/pigeons/' + str(n_inds) + '_birds/' + foldername + '/' + 'alltracks.csv')
df.head()

#### Rescale trajectory data to match the timescale of directional correlations while retaining pairwise nature of the measures

In [17]:
for fr in range(int(window_size)*fps//2, int(df['#t(centisec)'].max() - int(window_size)*fps//2)):
    tmp = df[(df['#t(centisec)'] >= fr - int(window_size)*fps//2) & (df['#t(centisec)'] <= fr + int(window_size)*fps//2)]

    tmp1 = tmp.loc[:,["f_id", "n_id", "dist", "speed_diff", "acc_diff"]]
    tmp2 = tmp.loc[:,["f_id", "n_id", "ang_pos_x", "ang_pos_y"]]

    tmp_ord1 = tmp1.groupby(['f_id', 'n_id']).median()
    tmp_ord1 = tmp_ord1.reset_index()
    tmp_ord1 = tmp_ord1.rename(index=str, columns={'dist':'dist', 'speed_diff':'speed_diff', 'acc_diff':'acc_diff'})
    tmp_ord2 = tmp2.groupby(['f_id', 'n_id']).mean()
    tmp_ord2 = tmp_ord2.reset_index()
    tmp_ord2 = tmp_ord2.rename(index=str, columns={'ang_pos_x':'ang_pos_x', 'ang_pos_y':'ang_pos_y'})
    
    tmp_ord = pd.merge(tmp_ord1, tmp_ord2, how='left')
    tmp_ord['#t(centisec)'] = fr
    
    df_rescaled = tmp_ord if fr == int(window_size)*fps//2 else df_rescaled.append(tmp_ord)
        
df_rescaled.head()

,f_id,n_id,dist,speed_diff,acc_diff,ang_pos_x,ang_pos_y,#t(centisec)
0,0,0,0.000000,0.000000,0.000000,NaN,NaN,150
1,0,1,7.452045,0.153359,-0.009048,-0.156380,-0.096019,150
2,0,2,4.197391,0.134417,-0.016236,-0.193548,-0.085239,150
3,0,3,4.422501,0.402323,0.027554,-0.192663,0.136224,150
4,0,4,4.159805,0.443118,-0.571681,-0.052083,-0.213041,150


#### Rescale trajectory data to match timescale of directional correlations and get individual level measures

In [18]:
for fr in range(int(window_size)*fps//2, int(df['#t(centisec)'].max() - int(window_size)*fps//2)):
    tmp = df[(df['#t(centisec)'] >= fr - int(window_size)*fps//2) & (df['#t(centisec)'] <= fr + int(window_size)*fps//2)]

    tmp1 = tmp.loc[:,["f_id", "speed", "acceleration", "dev_gspeed", "dev_gacc"]]
    tmp2 = tmp.loc[:,["f_id", "rotated_x", "rotated_y"]]

    tmp_ord1 = tmp1.groupby(['f_id']).median()
    tmp_ord1 = tmp_ord1.reset_index()
    tmp_ord1 = tmp_ord1.rename(index=str, columns={'speed':'speed', 'acceleration':'acceleration', 'dev_gspeed':'dev_gspeed', 'dev_gacc':'dev_gacc'})
    tmp_ord2 = tmp2.groupby(['f_id']).mean()
    tmp_ord2 = tmp_ord2.reset_index()
    tmp_ord2 = tmp_ord2.rename(index=str, columns={'rotated_x':'rx', 'rotated_y':'ry'})
    
    tmp_ord = pd.merge(tmp_ord1, tmp_ord2, how='left')
    tmp_ord['#t(centisec)'] = fr
    
    df2 = tmp_ord if fr == int(window_size)*fps//2 else df2.append(tmp_ord)
        
df2.head()

,f_id,speed,acceleration,dev_gspeed,dev_gacc,rx,ry,#t(centisec)
0,0,9.449549,4.620616,-0.243411,-0.003783,0.038930,-0.202239,150
1,1,9.571283,4.243177,0.040898,-0.005618,-2.455753,0.449590,150
2,2,9.532832,4.319635,-0.076442,-0.014211,-0.240393,-0.207771,150
3,3,9.840300,4.266960,-0.012301,-0.002535,0.121400,0.830338,150
4,4,8.749629,4.178941,0.047937,-0.173310,0.061512,-2.158862,150


In [19]:
del tmp
del tmp1
del tmp2
del tmp_ord1
del tmp_ord2
del tmp_ord
gc.collect()

2667

### Handle directional correlation data
Below, we binarise pairwise output obtained from the directional correlation cuda code. We then convert this measure of leadership per individual per frame.

#### Binarise pairwise events to 'leadership' or 'not leadership'

In [3]:
dcorr = pd.read_csv("/home/user/Documents/Vivek/cuda/DirectionalCorrelation/Data/Output/pigeons/" + str(n_inds) + "_birds/" + foldername + '/cross_correlation_' + window_size + '.csv')
dcorr.columns = ['#t(centisec)', 'f_id', 'n_id', 'tau', 'cc']

dcorr.loc[dcorr['cc'] < np.sqrt(3)/2,'cc'] = 0
dcorr.loc[np.abs(dcorr['tau']) <= 5/3,'tau'] = 0

@numba.njit(fastmath=True, parallel=True)
def binarise_leadership(frame, f_id, tau, cc):
    cc_out = np.zeros(cc.shape)
    frs = np.unique(frame)
    f_ids = np.unique(f_id)
    for fr in numba.prange(frs.shape[0]):
        for idx in numba.prange(f_ids.shape[0]):
            subset_idxs = np.where((frame == frs[fr]) & (f_id == f_ids[idx]) & (tau < 0))[0]
            if len(subset_idxs) > 0:
                ccvals = cc[subset_idxs]
                if ccvals.max() > np.sqrt(3.)/2.:
                    max_idx = subset_idxs[np.where(ccvals == ccvals.max())[0]]
                    cc_out[max_idx] = 1
    
    return cc_out

In [4]:
frame = dcorr['#t(centisec)'].values
f_id = dcorr['f_id'].values
tau = dcorr['tau'].values
cc = dcorr['cc'].values

dcorr['cc'] = binarise_leadership(frame, f_id, tau, cc)

In [5]:
dcorr.to_csv('/home/user/Documents/Vivek/cuda/DirectionalCorrelation/Data/Output/pigeons/' + str(n_inds) + '_birds/' + foldername + '/' + 'leadership_' + window_size + '.csv', mode='w')

#### Convert 'leadership' to an individual level metric over time (rather than a pairwise metric)

In [ ]:
@numba.njit(fastmath=True, parallel=True)
def individual_leadership(frame, f_id, n_id, cc):
    fr_out = np.zeros((np.unique(frame).shape[0]*n_inds,), dtype=np.int64)
    fid_out = np.zeros((np.unique(frame).shape[0]*n_inds,), dtype=np.int64)
    lscore = np.zeros((np.unique(frame).shape[0]*n_inds,), dtype=np.float64)
    
    frs = np.unique(frame)
    f_ids = np.unique(f_id)
    for fr in numba.prange(frs.shape[0]):
        for idx in numba.prange(f_ids.shape[0]):
            lead_idxs = np.where((frame == frs[fr]) & (n_id == f_ids[idx]))[0]
            follow_idxs = np.where((frame == frs[fr]) & (f_id == f_ids[idx]))[0]
            
            fr_out[fr*n_inds + idx] = frs[fr]
            fid_out[fr*n_inds + idx] = f_ids[idx]
            lscore[fr*n_inds + idx] = np.sum(cc[lead_idxs]) - np.sum(cc[follow_idxs])
            
    return fr_out, fid_out, lscore

In [ ]:
frame = dcorr['#t(centisec)'].values
f_id = dcorr['f_id'].values
n_id = dcorr['n_id'].values
cc = dcorr['cc'].values

fr_out, fid_out, lscore = individual_leadership(frame, f_id, n_id, cc)

In [ ]:
dcorr2 = pd.DataFrame(np.array([fr_out, fid_out, lscore]).T, columns=['#t(centisec)', 'f_id', 'lscore'])
dcorr2['lscore'] /= n_inds

tmp = dcorr2.loc[:,["f_id", "lscore"]]
tmp_ord = tmp.groupby(['f_id']).sum()
tmp_ord = tmp_ord.reset_index()
tmp_ord = tmp_ord.rename(index=str, columns={'lscore':'lfinal'})

dcorr2 = pd.merge(dcorr2, tmp_ord, how='left')
dcorr2.head()

### Create final datasets - individual and pairwise
These two dataframes will contain all data about morphology, movement, vision and leadership. The only way the two dataframes differ is based on whether the data is presented at the individual or pairwise level.

#### Individual data

In [25]:
df = pd.merge(df2, dcorr2)

cols = ['#t(centisec)', 'f_id']
df[cols] = df[cols].astype(np.int32)

df.head()

,f_id,speed,acceleration,dev_gspeed,dev_gacc,rx,ry,#t(centisec),lscore,lfinal
0,0,9.449549,4.620616,-0.243411,-0.003783,0.038930,-0.202239,150,0.0,-475.0
1,1,9.571283,4.243177,0.040898,-0.005618,-2.455753,0.449590,150,0.0,-124.5
2,2,9.532832,4.319635,-0.076442,-0.014211,-0.240393,-0.207771,150,0.1,-455.9
3,3,9.840300,4.266960,-0.012301,-0.002535,0.121400,0.830338,150,-0.1,-74.4
4,4,8.749629,4.178941,0.047937,-0.173310,0.061512,-2.158862,150,0.0,-64.8


In [26]:
df.to_csv('/home/user/Documents/Vivek/cuda/DirectionalCorrelation/Data/Output/pigeons/' + str(n_inds) + '_birds/' + foldername + '/' + 'individual_' + window_size + '.csv', mode='w')

#### Pairwise data

In [27]:
df = pd.merge(df_rescaled, dcorr)

cols = ['#t(centisec)', 'f_id', 'n_id', 'tau']
df[cols] = df[cols].astype(np.int32)

df.head()

,f_id,n_id,dist,speed_diff,acc_diff,ang_pos_x,ang_pos_y,#t(centisec),tau,cc
0,0,1,7.452045,0.153359,-0.009048,-0.156380,-0.096019,150,8,0.0
1,0,2,4.197391,0.134417,-0.016236,-0.193548,-0.085239,150,2,0.0
2,0,3,4.422501,0.402323,0.027554,-0.192663,0.136224,150,-2,0.0
3,0,4,4.159805,0.443118,-0.571681,-0.052083,-0.213041,150,-5,0.0
4,0,5,3.881257,0.784452,0.009779,0.116579,-0.293543,150,0,0.0


In [28]:
df['ang_pos'] = np.arctan2(df['ang_pos_y'], df['ang_pos_x'])
df['ang_pos'] = np.abs(df['ang_pos'])

df.drop(['ang_pos_x', 'ang_pos_y'], axis=1, inplace=True) 

In [29]:
df.to_csv('/home/user/Documents/Vivek/cuda/DirectionalCorrelation/Data/Output/pigeons/' + str(n_inds) + '_birds/' + foldername + '/' + 'pairwise_' + window_size + '.csv', mode='w')